# Experimentos y Análisis de resultados:

## Paso 1: Modelos Nuevos vs Random Player. Dos casos: Antes y después de buscar hiperparámetros

Se comparan 2 modelos que fueron entrenados 1 millon de timesteps:
* Modelo rojo: Valores mayormente por defecto y numero de steps = 256 por rollout. Se usan 8 environments, es decir: 256 * 8 = 2048 steps por epoch, usando 10 epochs por buffer de experiencias.
* Modelo celeste: Después de mejorar hiper parámetros se obtiene mayor reward, un clip fraction y divergencia KL menores, y la explained variance tiene mas sentido:
    * Incremento de num steps a 2048 * 8 envs = 16k steps por epoch
    * Incremento de 10 a 20 epochs por iteración
    * Incremento de batch size de 64 a 128
    * Disminución de learning rate de 0.0003 a 0.0002

#### Rewards:

![title](images/Analysis/Vs_Random_Reward.png)

Curva celeste siempre por encima de la roja

#### Losses:

![title](images/Analysis/Vs_Random_Loss.png)

Policy gradient loss: El celeste es mas cercano a 0. Interpreto que los ratios de policy / old_policy son mucho menores y por eso se manejan magnitudes mas bajas, y lo tomo como una mejora respecto al rojo (ver clip fraction y divergencia KL abajo).

Value Loss: El rojo es menor, pero hay que tener en cuenta que son menores los rewards acumulados también.

#### Clip Fraction y KL Divergence:

![title](images/Analysis/Vs_Random_Clip_KL.png)

Clip fraction y divergencia KL bastante inferiores en la celeste. Lo asocio a pasos mas pequeños donde la nueva policy es más parecida a la anterior que en la roja.

#### Explained Variance:

![title](images/Analysis/Vs_Random_Var.png)

La explained variance de la roja no tiene sentido (oscila en valores positivos y negativos). La celeste se mantiene siempre encima de 0 y más estable, con lo cual la estimación del value debería ser mejor.

## Paso 2: Tercera Variante contra Random. Achicar la red, entrenar mas, mejorar aprox de Value

* Redes mas chicas para policy y value (32 x 32 en vez de default 64 x 64). Siempre MLP
* Clipping sobre value.
* 2 millones de timesteps
* Buscaba estimar mejor la value para subir el explained variance.
* Nueva curva: Rosa

#### Rewards:

![title](images/Analysis/3rd_model_reward.png)

El reward parece mas inestable que en los casos anteriores, sube mas lento que el celeste, pero termina ubicandose por encima y llega a un mejor modelo en los picos (toca 100% de victorias). También fue entrenado mas tiempo.

#### Losses:

![title](images/Analysis/3rd_model_losses.png)

La policy loss de la rosa se acerca a 0 a medida que pasan los timesteps. El ratio debe ser cada vez mas chico.

La value loss de la rosa es bastante inferior y mas estable que en los casos anteriores.

#### Explained Variance:

![title](images/Analysis/3rd_model_var.png)

La explained variance parece estar por encima que en los casos anteriores y ser siempre positiva despues del "warm-up"

## Paso 3: Modelo nuevo (de cero) pero contra Random, Greedy y los 3 anteriores a la vez

La forma de implementarlo fue mediante la opción local_player="multiple" que agregué en CustomReversiModel (reversi_model.py), que permite pasar una carpeta en "path_local_player" en vez de un archivo. Todos los modelos que estén en esa carpeta + Greedy y Random son creados y repartidos entre los diferentes environments paralelos (cada env tiene un adversario diferente). En el buffer de experiencias quedan entonces mezcladas jugadas contra diferentes algoritmos.

De esta manera el modelo aprende a jugar contra muchas estrategias diferentes al mismo tiempo.

En el script "learn.py" se puede ver el uso de esta opción.

Aclaraciones:
* Nueva curva: Verde
* Los modelos adversarios estan en modo NO deterministico, para que haya mayor entropía y mas exploración. Si en evaluación tanto el modelo como adversario son deterministicos no entrena (gana o pierde siempre)

#### Rewards:

![title](images/Analysis/Vs_Multiple_reward.png)

La reward del verde es primero negativa y mucho mas inestable que las otras. Considerar que es un modelo nuevo que juega contra diferentes estrategias que son mucho mejores que random y mas diversas. Luego se estabiliza en valores similares, pero esta vez jugando contra modelos mejores.

#### Clip fraction y Divergencia KL:

![title](images/Analysis/Vs_Multiple_Clip_KL.png)

El clip version y la divergencia KL arrancan superiores al modelo celeste pero terminan siendo los mas bajos (lo asocio a un entrenamiento mas estable y el doble de timesteps).

#### Losses:

![title](images/Analysis/Vs_Multiple_Loss.png)

Para la loss de policy, el verde llega aun mas cerca de 0 que los anteriores (menor ratio entre new y old policy?)

En el caso de la loss de value, pareciera que el nuevo modelo sabe estimar bastante mejor ya que se llega a una menor loss.
Algo que noto es que la loss de la value primero baja, luego sube y finalmente baja en forma permanente y mas estable, a diferencia de los anteriores que solo bajaban. Atribuyo esta subida a la inestabilidad inicial en el mean reward (mayor diferencia entre lo estimado y lo real), hasta que se estabiliza.

#### Explained Variance:

![title](images/Analysis/Vs_Multiple_Var.png)

El explained variance de la verde es siempre superior a las otras curvas, lo cual debería indicar que el nuevo modelo estima mejor el valor de cada estado.

## Paso 4: Entrenamiento iterativo de modelo definitivo

El procedimiento realizado es:
* 1 - Entrenar varios modelos nuevos contra random (pasos 1 y 2 descritos arriba).
* 2 - Entrenar un modelo nuevo contra los anteriores con la opción "multiple" descrita arriba (paso 3).
* 3 - Jugar un torneo (ver tournament.py, play_tournament.py y la notebook 010_Torneo) y quedarme con el mejor.
* 4 - Cargar el mejor modelo y entrenarlo contra si mismo y los siguientes mejores modelos según resultados del torneo (2do, 3ro, etc). El objetivo de entrenar contra varios modelos es meter un poco de entropía y evitar caer en un óptimo local.
* 5 - Repetir pasos 3 y 4
* 6 - El resultado es el modelo final entrenado para competir

El siguiente es un ejemplo luego de varias iteraciones del procedimiento anterior. El nuevo modelo es el naranja:

![title](images/Analysis/Final_model_reward.png)

![title](images/Analysis/Final_model_clip_kl.png)

![title](images/Analysis/Final_model_var.png)

![title](images/Analysis/Final_model_losses.png)

Observaciones:
* Modelo que ya arranca entrenado (fui cargando modelos anteriores). Los primeros rewards estan por encima de 0.7 por ese motivo.
* Reward: Mucho mas inestable e inferior => Pero considerar que juega contra muchos modelos buenos a la vez!
* Clip Fraction y KL: Menores a los modelos anteriores. Menos cambios.
* Explained variance: Bastante estable. Similar a la convergencia del verde.
* Losses: Policy mas cercano a 0 => Menos ratio entre policies nueva y vieja. Value converge a un valor superior a las curvas rosas y verde.
* A partir de 3M de iteraciones parece no mejorar en ninguna métrica.

## Ejercicio Adicional: CNN

* Se realiza un entrenamiento (3 millones de timesteps) contra Random utilizando un features extractor con una red CNN, que servirá como features de entrada a un MLP de 32x32 (redes de policy y value de PPO), y es comparado con utilizar solo un flatten como features extractor y luego directo la red 32x32.
* Curva azul es MLP 32x32 solo, curva gris es CNN + MLP.
* Arquitectura de la red CNN:

        nn.Conv2d(1, 32, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.Conv2d(32, 16, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.Conv2d(16, 8, kernel_size=4, stride=1),
        nn.ReLU(),
        nn.Flatten()
        

* Ver: CNNFeaturesExtractor en custom_features_extractor.py


![title](images/Analysis/CNN_reward.png)

![title](images/Analysis/CNN_clip_KL.png)

![title](images/Analysis/CNN_var.png)

![title](images/Analysis/CNN_losses.png)

Observaciones:
* Si bien las métricas de explained variance y value loss son mejores en CNN, el mean reward pareciera ser más ruidoso. El valor conseguido contra Random es bastante peor que en el caso de solo MLP y converge mas lento.
* La versión con CNN tiene mas parámetros a entrenar que la MLP. Además de los 32x32 propios de las redes de value y policy (que yo elegí en ambos casos), se agregan 3 capas convolucionales + 1 lineal que adapta los canales de salida de la CNN a la dimensión esperada por PPO (observation_space). Considero que una red mas grande necesita más volumen de experiencias e iteraciones para converger. En un primer experimento de achicar la red de 64x64 a 32x32 funcionó mejor la segunda.

### Nota: Los modelos y los logs se pueden encontrar en las carpetas: selected_models y selected_logs